In [1]:
import multiprocessing
from GCForest import gcForest
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import Imputer
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
#读取Y
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y['2cArousal'] = 0
all_df_y['2cArousal'][all_df_y['valence'] >= 5] = 1
all_df_y['2cValence'] = 0
all_df_y['2cValence'][all_df_y['valence'] >= 5] = 1
print(all_df_y.head(5))
#读取32个通道的EEG数据，每个通道包含32×40=1280个信号样本（人次×每人次40实验）
#每个样本向量大小为8064点（63s*128Hz）
for eegCH in range(1,33,1):
    file_path = "./dump_file/CH{}_df_EEG_x".format(eegCH)
    locals()['CH{}_df_EEG_x'.format(eegCH)] = pickle.load(open(file_path,"rb"))
    
y = all_df_y[["2cValence"]]
#y = all_df_y[["2cArousal"]]
for seed in [0,100,200,300,400,500,600,700,800,900]:
    print("[seed:{}]****************************************************".format(seed))
    xTrainIdx = pickle.load(open("./dump_file/xTrainIdx_{}".format(seed),"rb"))
    xTestIdx = pickle.load(open("./dump_file/xTestIdx_{}".format(seed),"rb"))
    y_tr = y.loc[xTrainIdx]
    y_te = y.loc[xTestIdx]
    for eegCH in range(1,33,1): 
        #时域数据
        locals()['CH{}TrainSet'.format(eegCH)] = locals()['CH{}_df_EEG_x'.format(eegCH)].loc[xTrainIdx]
        locals()['CH{}TestSet'.format(eegCH)] = locals()['CH{}_df_EEG_x'.format(eegCH)].loc[xTestIdx]
    myWindowsSize = 256
    myStrideSize = 64
    gcf = gcForest(shape_1X=8064, window=myWindowsSize, stride=myStrideSize,tolerance=0.0,n_cascadeRF=1, 
                   min_samples_mgs=0.1, min_samples_cascade=0.1,n_jobs=19)
    if True:
        for eegCH in range(1,33,1): 
            print("CH{} running multi-grain scan".format(eegCH))
            xTrain,yTrain = locals()['CH{}TrainSet'.format(eegCH)].values,y_tr.values
            xTest = locals()['CH{}TestSet'.format(eegCH)].values   
            locals()['CH{}mgsTrainVector'.format(eegCH)] = gcf.mg_scanning(xTrain,yTrain)
            locals()['CH{}mgsTestVector'.format(eegCH)] = gcf.mg_scanning(xTest)
            filePath = "./dump_file/CH{}mgsTrainVector_{}_{}_{}".format(eegCH,myWindowsSize,myStrideSize,seed)
            pickle.dump(locals()['CH{}mgsTrainVector'.format(eegCH)],open(filePath,"wb"))
            filePath = "./dump_file/CH{}mgsTestVector_{}_{}_{}".format(eegCH,myWindowsSize,myStrideSize,seed)
            pickle.dump(locals()['CH{}mgsTestVector'.format(eegCH)],open(filePath,"wb"))
    else:
        for eegCH in range(1,33,1):
            filePath = "./dump_file/CH{}mgsTrainVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
            locals()['CH{}mgsTrainVector'.format(eegCH)] = pickle.load(open(filePath,"rb"))
            filePath = "./dump_file/CH{}mgsTestVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
            locals()['CH{}mgsTestVector'.format(eegCH)]= pickle.load(open(filePath,"rb"))
    X_tr_vector = locals()['CH{}mgsTrainVector'.format(1)]
    X_te_vector = locals()['CH{}mgsTestVector'.format(1)]
    for eegCH in range(2,33,1):
        nextTrainVector = locals()['CH{}mgsTrainVector'.format(eegCH)]
        nextTestVector = locals()['CH{}mgsTestVector'.format(eegCH)]
        X_tr_vector = np.concatenate([X_tr_vector,nextTrainVector],axis=1)
        X_te_vector = np.concatenate([X_te_vector,nextTestVector],axis=1)
    print(X_tr_vector.shape)
    print(X_te_vector.shape)
    #有缺失值，填充下
    X_tr_vector_fillna= pd.DataFrame(X_tr_vector).fillna(0).values
    X_te_vector_fillna= pd.DataFrame(X_te_vector).fillna(0).values
    _ = gcf.cascade_forest(X_tr_vector_fillna, y_tr)
    pred_proba = gcf.cascade_forest(X_te_vector_fillna)
    tmp = np.mean(pred_proba, axis=0)
    preds = np.argmax(tmp, axis=1)
    print("ACC",accuracy_score(y_true=y_te, y_pred=preds))
    print("F1",f1_score(y_true=y_te, y_pred=preds))
    print("Recal",recall_score(y_true=y_te, y_pred=preds))
    print("Precision",precision_score(y_true=y_te, y_pred=preds))

       valence  arousal  dominance  liking  2cArousal  2cValence
s01_0     7.71     7.60       6.90    7.83          1          1
s01_1     8.10     7.31       7.28    8.47          1          1
s01_2     8.58     7.54       9.00    7.08          1          1
s01_3     4.94     6.01       6.12    8.06          0          0
s01_4     6.96     3.92       7.19    6.05          1          1
[seed:0]****************************************************
CH1 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH2 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH3 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH4 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH5 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH6 running multi-grain scan
Slicing Sequence...


Slicing Sequence...
CH8 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH9 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH10 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH11 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH12 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH13 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH14 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH15 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH16 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH17 running multi-grain scan
Slicing Sequence...
Training MGS Random For

Training MGS Random Forests...
Slicing Sequence...
CH19 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH20 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH21 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH22 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH23 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH24 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH25 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH26 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH27 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH28 running multi-grain scan
Slicing Se

Slicing Sequence...
CH29 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH30 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH31 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH32 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
(896, 15744)
(384, 15744)
Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.6555555555555556
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.6555555555555556
ACC 0.6588541666666666
F1 0.7532956685499058
Recal 0.9216589861751152
Precision 0.6369426751592356
[seed:700]****************************************************
CH1 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH2 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH3 running multi-grain scan

CH4 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH5 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH6 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH7 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH8 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH9 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH10 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH11 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH12 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence...
CH13 running multi-grain scan
Slicing Sequence...
Training MGS Random Forests...
Slicing Sequence